In [2]:
%cd /mnt/c/Users/user/Desktop/과제/binfo1-work

/mnt/c/Users/user/Desktop/과제/binfo1-work


/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!conda install -y subread

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [7]:
!featureCounts -a gencode.vM27.annotation.gtf -o read-counts.txt *.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 10 BAM files                                     ||
||                                                                            ||
||                           CLIP-35L33G.bam                                  ||
||                           CLIP-let7d.bam                                   ||
||                           CLIP-let7f-1.bam                    

In [3]:
import pandas as pd
cnts = pd.read_csv('read-counts.txt', sep='\t', comment='#')
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam
0,ENSMUSG00000102693.2,chr1,3143476,3144545,+,1070,0,0,0,0,0,0,0,0,0,0
1,ENSMUSG00000064842.3,chr1,3172239,3172348,+,110,0,0,0,0,0,0,0,0,0,0
2,ENSMUSG00000051951.6,chr1;chr1;chr1;chr1;chr1;chr1;chr1,3276124;3276746;3283662;3283832;3284705;349192...,3277540;3277540;3285855;3286567;3287191;349212...,-;-;-;-;-;-;-,6094,4,0,0,0,1,1,1,0,0,0
3,ENSMUSG00000102851.2,chr1,3322980,3323459,+,480,3,0,0,0,0,0,0,0,0,0
4,ENSMUSG00000103377.2,chr1,3435954,3438772,-,2819,0,0,0,0,0,0,0,0,0,0


In [4]:
cnts['clip_enrichment'] = cnts['CLIP-35L33G.bam'] / cnts['RNA-control.bam']
cnts['rden_change'] = (cnts['RPF-siLin28a.bam'] / cnts['RNA-siLin28a.bam']) / (cnts['RPF-siLuc.bam'] / cnts['RNA-siLuc.bam'])
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change
0,ENSMUSG00000102693.2,chr1,3143476,3144545,+,1070,0,0,0,0,0,0,0,0,0,0,NaN,NaN
1,ENSMUSG00000064842.3,chr1,3172239,3172348,+,110,0,0,0,0,0,0,0,0,0,0,NaN,NaN
2,ENSMUSG00000051951.6,chr1;chr1;chr1;chr1;chr1;chr1;chr1,3276124;3276746;3283662;3283832;3284705;349192...,3277540;3277540;3285855;3286567;3287191;349212...,-;-;-;-;-;-;-,6094,4,0,0,0,1,1,1,0,0,0,4.0,NaN
3,ENSMUSG00000102851.2,chr1,3322980,3323459,+,480,3,0,0,0,0,0,0,0,0,0,inf,NaN
4,ENSMUSG00000103377.2,chr1,3435954,3438772,-,2819,0,0,0,0,0,0,0,0,0,0,NaN,NaN


In [10]:
#GOterm 찾는 것 부터 내가 수행
!pip install biopython
!pip install bioservices

In [56]:
!pip install biomart
!pip install concurrent

ERROR: Could not find a version that satisfies the requirement concurrent (from versions: none)
ERROR: No matching distribution found for concurrent


In [67]:
'''
#이것도 왠지 모르겠지만 잘 안돌아가서 폐기
from biomart import BiomartServer

def get_go_terms(gene_id):
    # Biomart 서버에 연결
    server = BiomartServer("http://www.ensembl.org/biomart")
    
    # Ensembl 데이터베이스에 액세스하는 데이터셋 선택
    dataset = server.datasets['mmusculus_gene_ensembl']
    
    # 필요한 필드 선택
    fields = ['ensembl_gene_id', 'go_id', 'name_1006']
    
    # 필터 설정: 유전자 ID로 필터링
    filters = {'ensembl_gene_id': gene_id}
    
    # Biomart를 사용하여 데이터 조회
    response = dataset.search({'filters': filters, 'attributes': fields})
    
    # 결과 처리
    go_terms = []
    for row in response.iter_lines():
        row_data = row.decode('utf-8').split('\t')
        go_terms.append((row_data[1], row_data[2]))  # GO ID와 GO 용어를 가져옴
    
    return go_terms
'''

'\n#이것도 왠지 모르겠지만 잘 안돌아가서 폐기\nfrom biomart import BiomartServer\n\ndef get_go_terms(gene_id):\n    # Biomart 서버에 연결\n    server = BiomartServer("http://www.ensembl.org/biomart")\n    \n    # Ensembl 데이터베이스에 액세스하는 데이터셋 선택\n    dataset = server.datasets[\'mmusculus_gene_ensembl\']\n    \n    # 필요한 필드 선택\n    fields = [\'ensembl_gene_id\', \'go_id\', \'name_1006\']\n    \n    # 필터 설정: 유전자 ID로 필터링\n    filters = {\'ensembl_gene_id\': gene_id}\n    \n    # Biomart를 사용하여 데이터 조회\n    response = dataset.search({\'filters\': filters, \'attributes\': fields})\n    \n    # 결과 처리\n    go_terms = []\n    for row in response.iter_lines():\n        row_data = row.decode(\'utf-8\').split(\'\t\')\n        go_terms.append((row_data[1], row_data[2]))  # GO ID와 GO 용어를 가져옴\n    \n    return go_terms\n'

In [57]:
!pip install Bio

In [7]:
from Bio import Entrez

Entrez.email = "your.email@example.com"

def fetch_go_id(go_id):
    handle = Entrez.esearch(db="gene", term=go_id)
    record = Entrez.read(handle)
    handle.close()
    return record

old_go_id = "GO:0005634"
result = fetch_go_id(old_go_id)
print(result)

{'Count': '6810', 'RetMax': '20', 'RetStart': '0', 'IdList': ['100259532', '3503598', '3503593', '3503543', '3503505', '3503443', '3503435', '3503430', '3503396', '3503354', '3503309', '3503305', '3503298', '3503243', '3503139', '3503121', '3503104', '3503007', '3502980', '3502965'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'GO[All Fields]', 'Field': 'All Fields', 'Count': '25065017', 'Explode': 'N'}, {'Term': '0005634[All Fields]', 'Field': 'All Fields', 'Count': '6812', 'Explode': 'N'}, 'AND', 'GROUP'], 'QueryTranslation': 'GO[All Fields] AND 0005634[All Fields]'}


In [37]:
import gzip

# GZIP 파일 읽기 및 필터링
with gzip.open("goa_mouse.gaf.gz", 'rt') as f:
    # GAF 파일을 데이터프레임으로 읽기
    mouse_data = pd.read_csv(f, sep='\t', comment='!', header=None)

# 열 이름 설정
mouse_data.columns = [
    'DB', 'DB_Object_ID', 'DB_Object_Symbol', 'Qualifier', 'GO_ID',
    'DB:Reference', 'Evidence_Code', 'With_From', 'Aspect', 'DB_Object_Name',
    'DB_Object_Synonym', 'DB_Object_Type', 'Taxon', 'Date', 'Assigned_By',
    'Annotation_Extension', 'Gene_Product_Form_ID'
]

# 데이터 출력 (예: 상위 5개 행)
print(mouse_data.head())

# 필요한 경우 데이터 저장
mouse_data.to_csv('mouse_go_annotations.gaf', sep='\t', index=False)

          DB DB_Object_ID DB_Object_Symbol     Qualifier       GO_ID  \
0  UniProtKB   A0A075B5I2            Trbv4   involved_in  GO:0007166   
1  UniProtKB   A0A075B5I2            Trbv4  is_active_in  GO:0005886   
2  UniProtKB   A0A075B5J4            Trbc2       part_of  GO:0042105   
3  UniProtKB   A0A075B5J6          Gm20730   involved_in  GO:0006955   
4  UniProtKB   A0A075B5J6          Gm20730  is_active_in  GO:0005615   

     DB:Reference Evidence_Code  \
0  GO_REF:0000033           IBA   
1  GO_REF:0000033           IBA   
2  GO_REF:0000033           IBA   
3  GO_REF:0000033           IBA   
4  GO_REF:0000033           IBA   

                                           With_From Aspect  \
0                 MGI:MGI:98608|PANTHER:PTN000588091      P   
1  MGI:MGI:98596|PANTHER:PTN000588091|UniProtKB:A...      C   
2              PANTHER:PTN001511926|UniProtKB:A0A5B9      C   
3  MGI:MGI:98936|PANTHER:PTN000587099|UniProtKB:P...      P   
4              PANTHER:PTN000587099|UniPr

In [5]:
cnts['Geneid'] = cnts['Geneid'].apply(lambda x: x.split('.')[0])

In [84]:
import numpy as np
cnts = cnts.dropna(subset=['clip_enrichment'])
cnts = cnts.dropna(subset=['rden_change'])
cnts = cnts[~cnts.isin([np.inf, -np.inf]).any(axis=1)]
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change
15,ENSMUSG00000102343,chr1;chr1;chr1;chr1;chr1,3975962;4054448;4055383;4055383;4056370,3976357;4054521;4056207;4055574;4056438,-;-;-;-;-,1364,67,0,0,0,31,207,157,1,2,0,2.161290,0.379227
19,ENSMUSG00000025902,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4561154;4561473;4561613;4561936;4562680;456268...,4562891;4562886;4562891;4562891;4563827;456289...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4772,6,0,0,0,1,8,5,2,3,0,6.000000,0.416667
21,ENSMUSG00000102269,chr1;chr1,4593128;4594669,4593826;4596960,+;+,2991,8,0,0,0,73,28,11,2,9,9,0.109589,0.087302
26,ENSMUSG00000098104,chr1,4758157,4759626,-,1470,29,0,0,0,63,26,15,2,7,0,0.460317,0.164835
30,ENSMUSG00000103922,chr1,4841354,4842422,+,1069,183,0,0,0,95,93,100,42,75,74,1.926316,0.602151


In [85]:
gene_IDs = cnts['Geneid'].tolist()
#print(gene_IDs)
print(len(gene_IDs))

19164


In [86]:
file_path = 'mouse_go_annotations.gaf'

# GAF 파일을 데이터프레임으로 읽기
gaf_data = pd.read_csv(file_path, sep='\t', comment='!', header=None)

# 열 이름 설정
gaf_data.columns = [
    'DB', 'DB_Object_ID', 'DB_Object_Symbol', 'Qualifier', 'GO_ID',
    'DB:Reference', 'Evidence_Code', 'With_From', 'Aspect', 'DB_Object_Name',
    'DB_Object_Synonym', 'DB_Object_Type', 'Taxon', 'Date', 'Assigned_By',
    'Annotation_Extension', 'Gene_Product_Form_ID'
]

def find_go_id(protein_id, target_GO_ids):
    # 입력된 gene ID에 해당하는 행 찾기
    matching_rows = gaf_data[gaf_data['DB_Object_ID'] == protein_id]
    
    if len(matching_rows) == 0:
        #print("Gene ID not found.")
        return []
    
    # 해당하는 행에서 GO ID 추출
    go_ids = matching_rows['GO_ID'].tolist()
    final_GO = []
    go_ids = list(set(go_ids))
    for i in range(len(go_ids)):
        if go_ids[i] in target_GO_ids:
            final_GO.append(go_ids[i])
    #print(go_ids)
    return final_GO

# Gene ID 입력 받기
#gene_id = input("Enter Gene ID: ")


/tmp/ipykernel_492/1544218344.py:4: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  gaf_data = pd.read_csv(file_path, sep='\t', comment='!', header=None)


In [87]:
!pip install requests

In [88]:
import requests

def query_biomart(ensembl_gene_ids):
    # Define the BioMart query
    xml_query = f"""
    <!DOCTYPE Query>
    <Query  virtualSchemaName = "default" formatter = "TSV" header = "1" uniqueRows = "1" count = "" datasetConfigVersion = "0.6" >
        <Dataset name = "mmusculus_gene_ensembl" interface = "default" >
            <Filter name = "ensembl_gene_id" value = "{','.join(ensembl_gene_ids)}"/>
            <Attribute name = "ensembl_gene_id" />
            <Attribute name = "uniprotswissprot" />
            <Attribute name = "uniprotsptrembl" />
        </Dataset>
    </Query>
    """
    
    # Send the query to the BioMart service
    response = requests.post('https://www.ensembl.org/biomart/martservice', data={'query': xml_query})
    response.raise_for_status()
    
    # Parse the response
    lines = response.text.strip().split('\n')
    headers = lines[0].split('\t')
    data = [line.split('\t') for line in lines[1:]]
    
    return headers, data

def get_uniprot_ids_for_genes(ensembl_gene_ids):
    headers, data = query_biomart(ensembl_gene_ids)
    
    gene_to_uniprot = {}
    for row in data:
        ensembl_gene_id = row[0]
        uniprot_ids = row[1:]  # Both Swiss-Prot and TrEMBL IDs
        uniprot_ids = [uid for uid in uniprot_ids if uid]  # Remove empty strings
        
        if ensembl_gene_id not in gene_to_uniprot:
            gene_to_uniprot[ensembl_gene_id] = set()
        
        gene_to_uniprot[ensembl_gene_id].update(uniprot_ids)
    
    # Convert sets to lists
    for gene_id in gene_to_uniprot:
        gene_to_uniprot[gene_id] = list(gene_to_uniprot[gene_id])
    
    return gene_to_uniprot

# Example Ensembl Gene IDs
#ensembl_gene_ids = ["ENSMUSG00000025902"]

# Get UniProt IDs for the given Ensembl Gene IDs
#gene_to_uniprot = get_uniprot_ids_for_genes(ensembl_gene_ids)
#print(gene_to_uniprot)

# Print the results
#for gene_id, uniprot_ids in gene_to_uniprot.items():
    #print(f"Gene ID: {gene_id}, UniProt IDs: {uniprot_ids}")


In [89]:
print(get_uniprot_ids_for_genes(gene_IDs))

{'ENSMUSG00000064341': ['P03888', 'Q4JFN6'], 'ENSMUSG00000064345': ['Q9MD59', 'P03893'], 'ENSMUSG00000064351': ['Q9MD68', 'P00397'], 'ENSMUSG00000064354': ['P00405', 'Q7JCZ1'], 'ENSMUSG00000064356': ['Q7JCZ0', 'P03930'], 'ENSMUSG00000064357': ['P00848', 'Q7JCY9'], 'ENSMUSG00000064358': ['P00416', 'Q7JCX7'], 'ENSMUSG00000064360': ['P03899', 'Q7GIP5'], 'ENSMUSG00000065947': ['P03903', 'Q9MD77'], 'ENSMUSG00000064363': ['Q7JCY6', 'P03911'], 'ENSMUSG00000064367': ['Q9MD82', 'P03921'], 'ENSMUSG00000064368': ['P03925', 'Q7JCY4'], 'ENSMUSG00000064370': ['P00158', 'Q7JCZ3'], 'ENSMUSG00000053211': ['P10925'], 'ENSMUSG00000069053': ['P31254', 'A0A9K3AWV9'], 'ENSMUSG00000056673': ['A0A087WR43', 'A0A9K3AWU9', 'Q8BQM3', 'Q62240'], 'ENSMUSG00000069049': ['Q9Z0N2', 'A0A9K3FHH7'], 'ENSMUSG00000068457': ['E9PWW8', 'D6RFT4', 'F7BBC1', 'D3Z2A3', 'E9Q504', 'Q3UQE8', 'G3X9F2', 'D6RJL0'], 'ENSMUSG00000069045': ['Q62095', 'A0A087WQV7', 'A0A9K3AWW9'], 'ENSMUSG00000024304': ['P15116', 'D3YYT0'], 'ENSMUSG0000004

In [90]:
G_P_id_dic = get_uniprot_ids_for_genes(gene_IDs)
print(len(G_P_id_dic))

14230


In [91]:
target_GO_terms = ['GO:0005634', 'GO:0031966', 'GO:0016021', 'GO:0005737', 'GO:0005739', 'GO:0005789', 
                   'GO:0030203', 'GO:0005576', 'GO:0009986', 'GO:0005794', 'GO:0005783', 
                   'GO:0005788', 'GO:0005509', 'GO:0015467', 'GO:0006334', 'GO:0000786']
Gene_GO_lists = []
for i in range(len(gene_IDs)):
    uniprot_id = G_P_id_dic.get(gene_IDs[i], [])
    for j in range(len(uniprot_id)):
        x = find_go_id(uniprot_id[j], target_GO_terms)
        if x != []:
            x.insert(0, gene_IDs[i])
            Gene_GO_lists.append(x)
            #print(x)
        
        
        '''
        for h in range(len(target_GO_terms)):
            if target_GO_terms[h] in x:
                print(f'ID : {uniprot_id[j]}, go : {target_GO_terms[h]}')
                '''

In [92]:
a = [item[0] for item in Gene_GO_lists]
b = [item[1:] for item in Gene_GO_lists]

df = pd.DataFrame({'Geneid': a, 'GO': b})
df.head()

,Geneid,GO
0,ENSMUSG00000025902,[GO:0005634]
1,ENSMUSG00000033845,[GO:0005739]
2,ENSMUSG00000025903,"[GO:0009986, GO:0005783, GO:0005739, GO:0005737]"
3,ENSMUSG00000033813,[GO:0005634]
4,ENSMUSG00000002459,"[GO:0005634, GO:0005794]"


In [93]:
final_data = pd.merge(cnts, df, left_on='Geneid', right_on='Geneid')
final_data.head()
#print(len(final_data))

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change,GO
0,ENSMUSG00000025902,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4561154;4561473;4561613;4561936;4562680;456268...,4562891;4562886;4562891;4562891;4563827;456289...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4772,6,0,0,0,1,8,5,2,3,0,6.000000,0.416667,[GO:0005634]
1,ENSMUSG00000033845,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4843429;4843434;4844659;4846600;4847748;484774...,4844739;4847024;4844739;4847024;4847871;484787...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,8487,4014,0,0,0,1355,3022,2563,1859,3191,3,2.962362,0.494091,[GO:0005739]
2,ENSMUSG00000025903,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4878011;4878046;4878053;4878119;4878121;487813...,4878205;4878205;4878205;4878205;4878205;487820...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;...,7145,2045,0,0,0,574,1219,1236,33,44,44,3.562718,0.760459,"[GO:0009986, GO:0005783, GO:0005739, GO:0005737]"
3,ENSMUSG00000033813,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4928037;4928261;4937693;4937693;4948250;494825...,4928199;4928726;4937755;4937755;4948355;494835...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+,3017,1658,0,0,0,762,2130,2225,328,609,607,2.175853,0.562609,[GO:0005634]
4,ENSMUSG00000002459,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4979799;4979799;4980625;4980697;4982537;498253...,4980885;4980885;4980885;4980885;4982771;498277...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4239,5,0,0,0,1,1,8,3,2,0,5.000000,12.000000,"[GO:0005634, GO:0005794]"


In [94]:
Data = final_data.drop_duplicates(subset='Geneid')
print(len(Data))

8893


In [95]:
def make_df(GO_term):
    type = Data[Data['GO'].apply(lambda x: GO_term in x and 'GO:0031966' not in x)]
    print(len(type))
    return type

In [96]:
nucleus = make_df('GO:0005634')
extracellular_region = make_df('GO:0005576')
endoplasmic_reticulum = make_df('GO:0005783')
mitochondrion = make_df('GO:0005739')
cytoplasm = make_df('GO:0005737')
endoplasmic_reticulum_membrane = make_df('GO:0005789')
glycosaminoglycan_metabolic_process = make_df('GO:0030203')
endoplasmic_reticulum_lumen = make_df('GO:0005788')
cell_surface = make_df('GO:0009986')
calcium_ion_binding = make_df('GO:0005509')
Golgi_apparatus = make_df('GO:0005794')
nucleosome = make_df('GO:0000786')
nuclosome_assembly = make_df('GO:0006334')

4004
482
742
1341
3808
696
5
87
402
428
685
85
60


In [97]:
nucleus.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change,GO
0,ENSMUSG00000025902,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4561154;4561473;4561613;4561936;4562680;456268...,4562891;4562886;4562891;4562891;4563827;456289...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4772,6,0,0,0,1,8,5,2,3,0,6.000000,0.416667,[GO:0005634]
3,ENSMUSG00000033813,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4928037;4928261;4937693;4937693;4948250;494825...,4928199;4928726;4937755;4937755;4948355;494835...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+,3017,1658,0,0,0,762,2130,2225,328,609,607,2.175853,0.562609,[GO:0005634]
4,ENSMUSG00000002459,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4979799;4979799;4980625;4980697;4982537;498253...,4980885;4980885;4980885;4980885;4982771;498277...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4239,5,0,0,0,1,1,8,3,2,0,5.000000,12.000000,"[GO:0005634, GO:0005794]"
6,ENSMUSG00000025907,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,6276421;6276451;6284869;6284887;6285232;629382...,6276494;6276494;6285181;6285181;6285476;629396...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;...,14690,1520,0,0,0,280,694,903,132,251,251,5.428571,0.684272,"[GO:0005634, GO:0005737]"
13,ENSMUSG00000042414,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1,13183681;13186004;13188987;13192585;13194587;1...,13184711;13186105;13189189;13192855;13194744;1...,-;-;-;-;-;-;-;-,2564,227,0,0,0,157,702,414,121,143,0,1.445860,0.499014,[GO:0005634]


In [108]:
#1
nucleus_clip = nucleus['clip_enrichment'].mean()
nucleus_rden = nucleus['rden_change'].mean()
print('Nucleus:', np.log2(nucleus_clip), np.log2(nucleus_rden))
#2
nucleosome_clip = nucleosome['clip_enrichment'].mean()
nucleosome_rden = nucleosome['rden_change'].mean()
print('Nucleosome:', np.log2(nucleosome_clip), np.log2(nucleosome_rden))
#3
extracellular_region_clip = extracellular_region['clip_enrichment'].mean()
extracellular_region_rden = extracellular_region['rden_change'].mean()
print('extracellular region:', np.log2(extracellular_region_clip), np.log2(extracellular_region_rden))
#4
cytoplasm_clip = cytoplasm['clip_enrichment'].mean()
cytoplasm_rden = cytoplasm['rden_change'].mean()
print('cytoplasm:', np.log2(cytoplasm_clip), np.log2(cytoplasm_rden))
#5
endoplasmic_reticulum_clip = endoplasmic_reticulum['clip_enrichment'].mean()
endoplasmic_reticulum_rden = endoplasmic_reticulum['rden_change'].mean()
print('endoplasmic reticulum:', np.log2(endoplasmic_reticulum_clip), np.log2(endoplasmic_reticulum_rden))
#6
endoplasmic_reticulum_membrane_clip = endoplasmic_reticulum_membrane['clip_enrichment'].mean()
endoplasmic_reticulum_membrane_rden = endoplasmic_reticulum_membrane['rden_change'].mean()
print('endoplasmic reticulum membrane:', np.log2(endoplasmic_reticulum_membrane_clip), np.log2(endoplasmic_reticulum_membrane_rden))
#7
mitochondrion_clip = mitochondrion['clip_enrichment'].mean()
mitochondrion_rden = mitochondrion['rden_change'].mean()
print('mitochondrion:', np.log2(mitochondrion_clip), np.log2(mitochondrion_rden))
#8
glycosaminoglycan_metabolic_process_clip = glycosaminoglycan_metabolic_process['clip_enrichment'].mean()
glycosaminoglycan_metabolic_process_rden = glycosaminoglycan_metabolic_process['rden_change'].mean()
print('glycosaminoglycan metabolic process:', np.log2(glycosaminoglycan_metabolic_process_clip), np.log2(glycosaminoglycan_metabolic_process_rden))
#9
endoplasmic_reticulum_lumen_clip = endoplasmic_reticulum_lumen['clip_enrichment'].mean()
endoplasmic_reticulum_lumen_rden = endoplasmic_reticulum_lumen['rden_change'].mean()
print('endoplasmic reticulum lumen:', np.log2(endoplasmic_reticulum_lumen_clip), np.log2(endoplasmic_reticulum_lumen_rden))
#10
calcium_ion_binding_clip = calcium_ion_binding['clip_enrichment'].mean()
calcium_ion_binding_rden = calcium_ion_binding['rden_change'].mean()
print('calcium ion binding:', np.log2(calcium_ion_binding_clip), np.log2(calcium_ion_binding_rden))
#11
Golgi_apparatus_clip = Golgi_apparatus['clip_enrichment'].mean()
Golgi_apparatus_rden = Golgi_apparatus['rden_change'].mean()
print('Golgi apparatus:', np.log2(Golgi_apparatus_clip), np.log2(Golgi_apparatus_rden))
#12
nuclosome_assembly_clip = nuclosome_assembly['clip_enrichment'].mean()
nuclosome_assembly_rden = nuclosome_assembly['rden_change'].mean()
print('nucleosome assembly:', np.log2(nuclosome_assembly_clip), np.log2(nuclosome_assembly_rden))
#13
cell_surface_clip = cell_surface['clip_enrichment'].mean()
cell_surface_rden = cell_surface['rden_change'].mean()
print('cell surface:', np.log2(cell_surface_clip), np.log2(cell_surface_rden))



Nucleus: 1.5336292374290867 -0.8022430806234688
Nucleosome: 5.162534160139761 -0.23344454987688484
extracellular region: 2.1674364055033166 -0.07798100933112885
cytoplasm: 1.2477517066999493 -0.7360958775303038
endoplasmic reticulum: 1.9715504547516887 -0.25128536035432697
endoplasmic reticulum membrane: 2.086609860494083 -0.14101199897698108
mitochondrion: 1.1147241729313888 -0.6716346739601325
glycosaminoglycan metabolic process: 1.5377576751811464 0.20466005463518902
endoplasmic reticulum lumen: 2.3723071856464975 -0.1509281167272724
calcium ion binding: 2.0038324687257196 -0.2405099536068566
Golgi apparatus: 1.8769974240806435 -0.18692034061931317
nucleosome assembly: 4.592545625774457 -0.7519366052600283
cell surface: 2.3591606933538114 -0.31148681732177996


[2.803921568627451, 4.907079646017699, 0.1, 2.0692307692307694, 0.9619220308250227, 0.8148148148148148, 4.285714285714286, 5.166666666666667, 76.5, 17.069565217391304, 0.7123287671232876, 1.7554347826086956, 2.0352422907488985, 1.6721915285451197, 0.8395061728395061, 1.0351222714590513, 1.5384615384615385, 0.4945054945054945, 1.8922413793103448, 22.363636363636363, 0.45, 0.5922848664688427, 0.6067415730337079, 1.85, 1.2232451093210588, 1.5174746335963922, 3.8703703703703702, 1.3076923076923077, 0.36727688787185353, 1.7863247863247864, 4.997105643994211, 191.0, 4.280373831775701, 4.0633608815427, 110.5, 206.5, 68.9, 84.0, 38.833333333333336, 23.0, 287.125, 36.0, 72.5, 3.8, 2.0, 1.0, 87.4, 105.33333333333333, 12.714285714285714, 7.555555555555555, 103.75, 38.4, 0.647887323943662, 16.866666666666667, 29.142857142857142, 145.0, 158.66666666666666, 4.333333333333333, 4.025, 16.166666666666668, 20.333333333333332, 22.782608695652176, 39.11764705882353, 13.7, 32.705882352941174, 64.75, 9.3333